In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from dolfin import *
from osgeo import gdal
import pygmsh
import meshio
import scipy.interpolate as interp
import meshio
from itkwidgets import view
import pyvista as pv
from pyvista import set_plot_theme
set_plot_theme('document')

### Notebook to play with simple 2D groundwater flow with shallow water and Bernouli's approximation

$$ f\frac{\partial h}{\partial t} = \nabla \cdot \left[Kh \nabla \left( h + z \right)\right] + S $$

* Boundary conditions are $ \partial h/\partial y = 0 $ on north and south, and $ h = const. $ on east and west.
* $z$ is the topography of the impermiable base layer.
* $f$ is the drainiable porosity.
* $K$ is conductivity.

In [ ]:
h_south = 10  # south boundary height
f = 0.34  # drainable porosity
S = 0.03*24/f  # source term mm/hr -> m/day
K = 0.0001*60*60*24/f  # conductivity m/s -> m/day
D = 20  # initial water height m

Build the model mesh based on the Sithas DEM

In [ ]:
# load in DEM
gdal_data = gdal.Open('/work/armitagj/code/caesar-explore/riu-bergantes/paleo-v1/input_data/paleoDEM_v1.tif')
data_array = gdal_data.ReadAsArray().astype(np.float)

# extract contour of the catchment (need to use three segments)
catchment = np.nan_to_num(data_array)
cs = plt.contour(catchment, [0])
plt.show()
xy = cs.collections[0].get_paths()[0]
coor_xy = xy.vertices
xy = cs.collections[0].get_paths()[2]
coor_xy = np.append(coor_xy, xy.vertices, axis=0)
xy = cs.collections[0].get_paths()[1]
coor_xy = np.append(coor_xy, xy.vertices, axis=0)

lx = np.max(coor_xy[:,0])
ly = np.max(coor_xy[:,1])  # characteristic length: ly
coor_xy_dimless = coor_xy/ly

# chop off the outflow to make a straight boundary
outflow = np.min(coor_xy_dimless[:,1]) + 0.02
coor_xy_dimless[np.where(coor_xy_dimless[:,1] < outflow), 1] = outflow

coor_z = np.zeros((len(coor_xy), 1))
coor = np.concatenate((coor_xy, coor_z), axis=1)

# create the mesh
geom = pygmsh.built_in.Geometry()
poly = geom.add_polygon(coor[0::100])
i = 0
for line in poly.line_loop.lines:
    geom.add_physical([poly.line_loop.lines[i]], i + 1)
    i = i + 1
geom.add_physical([poly.surface], i + 1)
res = 64
maxcell = ly/res
gmshmesh = pygmsh.generate_mesh(geom, extra_gmsh_arguments=["-2", "-clmax", str(maxcell)])

# export the mesh
cells = np.vstack(np.array([cells.data for cells in gmshmesh.cells
                            if cells.type == "triangle"]))
triangle_mesh = meshio.Mesh(points=gmshmesh.points,
                            cells=[("triangle", cells)])
facet_cells = np.vstack(np.array([cells.data for cells in gmshmesh.cells
                                  if cells.type == "line"]))
facet_data = gmshmesh.cell_data_dict["gmsh:physical"]["line"]
facet_mesh = meshio.Mesh(points=gmshmesh.points,
                         cells=[("line", facet_cells)],
                         cell_data={"name_to_read": [facet_data]})
# export
meshio.xdmf.write("mesh.xdmf", triangle_mesh)
meshio.xdmf.write("facet_mesh.xdmf", facet_mesh)

# read in the mesh for dolfin
mesh = Mesh()
with XDMFFile("mesh.xdmf") as infile:
    infile.read(mesh)
mvc = MeshValueCollection("size_t", mesh, 2)
with XDMFFile("facet_mesh.xdmf") as infile:
    infile.read(mvc, "name_to_read")
mf = cpp.mesh.MeshFunctionSizet(mesh, mvc)

# define function space
V = FunctionSpace(mesh, 'P', 1)
z = Function(V)

# Get the global coordinates
gdim = mesh.geometry().dim()
gc = V.tabulate_dof_coordinates().reshape((-1, gdim))

# Interpolate elevation into the initial condition
ny, nx = np.shape(data_array)
x, y = np.meshgrid(np.linspace(0, nx, nx),
                   np.linspace(0, ny, ny))

flat_data = data_array.flatten()
new_data = flat_data[~np.isnan(flat_data)]
flat_x = x.flatten()[~np.isnan(flat_data)]
flat_y = y.flatten()[~np.isnan(flat_data)]

elevation = interp.griddata((flat_x[0::100], flat_y[0::100]),
                            new_data[0::100],
                            (gc[:, 0], gc[:, 1]),
                            method='nearest')

z.vector()[:] = elevation-100  # bedrock elevation from topography
outpoint = np.min(gc[:, 1])
print(outflow)
print(outpoint)
print(np.min(x[1]))

# define boundary
class South(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], outpoint)

# apply boundary condition
uS = Constant(h_south)
South().mark(mf, 1)
bc = DirichletBC(V, z, mf, 1)

 Check bedrock

In [ ]:
File("mesh.pvd").write(mesh)
test = pv.read("mesh000000.vtu")
p = pv.Plotter(notebook=True, border=False)
p.add_mesh(test, show_edges=True)
p.show()

Try to get stable timestep for this...
$$ \Delta t \leq \frac{\left(\Delta x\right)^2}{2\kappa(h)} $$
the problem is the diffusion coefficient is a function of the hydraulic head. But if I assume the hydraulic head is a maximum of 100m


In [ ]:
hmax = 100  # meters
kappa = K*hmax
cfl = 0.1  # just to be safe
deltax = np.min([np.min(np.diff(gc[:, 0])), np.min(np.diff(gc[:, 1]))])
dt = cfl * deltax * deltax / (2 * kappa)
print('dt = {}; dx = {}'.format(dt, deltax))

Define the trial $u$ and test $v$ functions and the source term, $S$. I used [this](https://fenicsproject.org/pub/tutorial/sphinx1/._ftut1004.html) resource as a first try at the implicit solution.

In [ ]:
dt = 0.001

In [ ]:
def kappa(u):
    return K*u

# Define variational problem
u = Function(V)
u_n = Function(V)
u_n = interpolate(Constant(D), V)
v = TestFunction(V)
f = Constant(S)
f = Expression("f", f=f, degree=1)
k = Constant(K)

#F = u*v*dx + dt*inner(kappa(u) * nabla_grad(u + z), nabla_grad(v + z))*dx - (u_n + dt*f)*v*dx


Solve for the time dependent equation

In [ ]:
# time steps fr explicit solutions
itera = 0           # iteration counter
maxiter = 2         # max no of iterations allowed

while itera < maxiter:
    itera += 1
    solve(a == L, u, bc, solver_parameters={"newton_solver": {"relative_tolerance": 1e-6}})
    u_n.assign(u)

In [ ]:
# Save solution to file
file = File("timedep-groundwater-head.pvd")
u.rename("h","head")
file << u
file = File("timedep-groundwater-base.pvd")
z.rename("z","base")
file << z

Plot the solution

In [ ]:
head = pv.read('timedep-groundwater-head000000.vtu')
base = pv.read('timedep-groundwater-base000000.vtu')
head['groundwater'] = base['z'] + head['h']
warp = head.warp_by_scalar(scalars='groundwater',factor=1)
warpz = base.warp_by_scalar(scalars='z',factor=1)
# view(geometries=warp)

In [ ]:
p = pv.Plotter(notebook=False, border=False)
p.add_text("Steady State Solution", font_size=12)
p.add_mesh(warp, scalars='groundwater', cmap='viridis', lighting=True, show_scalar_bar=True, opacity=0.85, color=True)
p.add_mesh(warpz, scalars='z', cmap='magma', lighting=True, show_scalar_bar=True)
p.show_bounds(grid='front', font_size=8, location='outer', all_edges=True)
p.show(use_panel=False,)

In [ ]:
warph = head.warp_by_scalar(scalars='h',factor=10)

In [ ]:
p = pv.Plotter(notebook=False, border=False)
p.add_text("Steady State Solution", font_size=12)
p.add_mesh(warph, scalars='h', cmap='RdBu_r', lighting=True, show_scalar_bar=True)
p.show(use_panel=False,)